<img src="images/header.png">

<h1><center>Алгоритмы интеллектуальной обработки больших объемов данных</center></h1>
<hr>
<h2><center>Линейные модели (практика)</center></h2>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,8)

# Для кириллицы на графиках
font = {'family': 'Verdana',
        'weight': 'normal'}
plt.rc('font', **font)

In [ ]:
try:
    from ipywidgets import interact, IntSlider, fixed
except ImportError:
    print u'Так надо'

# Линейная регрессия

## Пример: Стоимость автомобиля

Загрузите [тренировочные данные](http://bit.ly/1gIQs6C) и [тестовые данные](http://bit.ly/IYPHrK) - уже знакомые нам данные по автомобилям.

In [ ]:
df_train = pd.read_csv('http://bit.ly/1gIQs6C')
df_test = pd.read_csv('http://bit.ly/IYPHrK')

In [ ]:
df_train.head()

In [ ]:
df_train.plot(x='mileage', y='price', kind='scatter', s=120)

Кажется, что между стоимостью и пробегом зависимость линейная - давайте ее найдем!

In [ ]:
X_train = df_train.mileage.values.reshape(-1, 1)
y_train = df_train.price.values

In [ ]:
from sklearn.linear_model import LinearRegression

Обучим модель

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
print 'Модель:\nprice = %.2f + (%.2f)*mileage' % (model.intercept_, model.coef_[0])

Нарисуйте предсказание модели (прямую) вместе с данными на плоскости. Здесь можно либо явно взять уравнение прямой и посчитать значения в каждой точке, либо через predict.

In [ ]:
df_train.plot(x='mileage', y='price', kind='scatter', s=120)

## Your Code Here

### Меры качества

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

Можно посчитать простые варианты агрегирования остатков, например:

* $\frac{1}{n} \sum_i |\hat{y}^{(i)}-y^{(i)}|$ - средняя абсолютная ошибка
* $\frac{1}{n} \sum_i (\hat{y}^{(i)}-y^{(i)})^2$ - средняя квадратичная ошибка

In [ ]:
y_hat = model.predict(X_train)

In [ ]:
print('Средняя абсолютная ошибка %.2f' % mean_absolute_error(y_train, y_hat))
print('Средняя квадратичная ошибка %.2f' % mean_squared_error(y_train, y_hat))

Можно рассмотреть более сложную меру: коэффициент детерминации $R^2$:

* $TSS = \sum_i (y^{(i)}-\bar{y})^2$ - общая сумма квадратов (total sum of squares)
* $RSS = \sum_i (\hat{y}^{(i)}-y^{(i)})^2$ - сумма квадратов остатков (residual sum of squares)
* $ESS = \sum_i (\hat{y}^{(i)}-\bar{y})^2$ - объясненная сумма квадратов (explained sum of squares)

Для простоты будем считать, что
$$TSS = ESS + RSS$$

Тогда Коэффициент детерминации $R^2=1-\frac{RSS}{TSS}$

Рассчитайте его для нашей модели


## Преобразование переменных

### Нормализация

Переход к близким или единым шкалам улучшает сходимость градиентного спуска, уменьшает риск переполнения разрядности чисел, однако приходится жертвовать прямой интерпретируемостью..

Нормализацию обычно проделывают для вещественных признаков.

Нормализация z-score:
1. Вычитаем среднее: $x - \bar{x}$
2. Делим на стандартное отклонение: $\frac{x - \bar{x}}{std(x)}$

Можно проделать вручную, можно с помошью метода ниже

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

model = LinearRegression(fit_intercept=True)
model.fit(X_train_scaled, y_train)

In [ ]:
print('Модель:\nprice = %.2f + (%.2f)*mileage`' % (model.intercept_, model.coef_[0]))

## Как обучать линейную регрессию?
Попробуем разобраться без всяких `.predict()` и `.fit()`

### Рассмотрим случай с одним признаком и свободным членом

$X$ - признаковое описание наблюдений,<br\> $y$ - прогнозируемая величина

Пусть задана функция ошибки (функция потерь) $L(\cdot)$. <br\>
Нам надо построить такой функционал $f(X)$, который будет выдавать значение наиболее близкие к $y$, иначе говоря: $$L\left(f(X) - y\right) \rightarrow\min $$

Определим функцию потерь, как сумму квадратов разности выдаваемого ответа функционала и реального значения: 
$$ L(\cdot) = \frac{1}{2n}\sum_{i=1}^n(f(x^{(i)}) - y^{(i)})^2$$

Так как среди всего множества моделей мы выбрали линейную регрессию, то $$f(X) = \beta_0 + \beta_1x_1$$
Подставляем это выражение в $L(\cdot)$ и находим $\beta_0$,
$\beta_1$!

$$ L(\beta_0,\beta_1) = \frac{1}{2n}\sum^{n}_{i=1}(f(x^{(i)})  - y^{(i)})^2 = \frac{1}{2n}\sum^{n}_{i=1}(\beta_0 + \beta_1x^{(i)}_1 - y^{(i)})^2  \rightarrow \min\limits_{\beta_0, \beta_1} $$

Изобразим функцию потерь на трехмерном графике в зависимости от $\beta_0$ и $\beta_1$ для задачи с автомобилем

In [ ]:
# для удобства добавим столбец из "1" в матрицу с признаком "пробег"
X_model = np.c_[np.ones(X_train.shape), X_train]
X_model.shape

In [ ]:
from mpl_toolkits import mplot3d

In [ ]:
beta0 = np.linspace(11000 , 13000, 100)
beta1 = np.linspace(-2450, -250, 100)

B0, B1= np.meshgrid(beta0, beta1)

B_all = np.c_[B0.reshape(-1,1), B1.reshape(-1,1)].T

L = X_model.dot(B_all) - y_train.reshape(-1,1)
L = L ** 2
L = L.mean(axis=0)/2
L = L.reshape(B0.shape)


fig = plt.figure(figsize=(14, 7))
ax = fig.add_subplot(1, 2, 1, projection='3d')
ax.view_init(40, 25)
ax.plot_surface(B0, B1, L, alpha=0.3,)
ax.set_xlabel('beta_0')
ax.set_ylabel('beta_1')

ax = fig.add_subplot(1, 2, 2)
contour = ax.contour(B0, B1, L)
plt.clabel(contour, inline=1, fontsize=10)
ax.set_xlabel('beta_0')
ax.set_ylabel('beta_1')

### Градиентный спуск

Градиентый спуск - это итеративный метод оптимизации функции. Он заключается в постепенном перемещении к точке экспетмума в направлении антиградиента этой функции в точке.

Посчитаем, чему равен градиент функции потерь $L(\beta_0, \beta_1):$
$$ \frac{\partial L}{\partial \beta_0} = \frac{1}{n}\sum^{n}_{i=1}(\beta_0 + \beta_1x^{(i)}_1 - y^{(i)})$$
$$ \frac{\partial L}{\partial \beta_1} = \frac{1}{n}\sum^{n}_{i=1}(\beta_0 + \beta_1x^{(i)}_1 - y^{(i)})x_1^{(i)}$$

Иногда проще это записать в виде матриц:
$$ \frac{\partial L}{\partial \beta} = X^\top(X\beta - y)$$

Метод градиентного спуска заключается в итеративном и **одновременном(!!!)** обновлении значений $\beta$ в направлении, противоположному градиенту:
$$ \beta := \beta -  \frac{\alpha}{n} \frac{\partial L}{\partial \beta}$$


Теперь к шагам алгоритма:

* Задаем случайное начальное значение для $\beta$
* Пока не будет достигнуто правило останова:
    * Считаем ошибку и значение функции потерь
    * Считаем градиент
    * Обновляем коэффициенты

In [ ]:
def gradient_descent(X, y, iters, alpha):
    
    costs = []
    betas = []
    
    n = y.shape[0] 
    Beta = np.random.rand(X.shape[1])
    for i in xrange(iters):
        y_hat = X.dot(Beta)
        
        # считаем ошибку и значение функции потерь
        
        # считаем градиент

        # обновляем коэффициенты
        # Beta = Beta - alpha/n * grad
                    
    return Beta, costs, betas

In [ ]:
Beta, costs, betas = gradient_descent(X_model, y_train, 100, 0.05)

In [ ]:
beta0 = np.linspace(11000 , 13000, 100)
beta1 = np.linspace(-2450, -250, 100)

B0, B1= np.meshgrid(beta0, beta1)

B_all = np.c_[B0.reshape(-1,1), B1.reshape(-1,1)].T

L = X_model.dot(B_all) - y_train.reshape(-1,1)
L = L ** 2
L = L.mean(axis=0)/2
L = L.reshape(B0.shape)

fig, ax = plt.subplots(1,1)
contour = ax.contour(B0, B1, L)
plt.clabel(contour, inline=1, fontsize=10)
ax.set_xlabel('beta_0')
ax.set_ylabel('beta_1')

betas = np.array(betas)
ax.plot(betas[:,0], betas[:,1], marker='*')


## Выбросы

Квадратичная ошибка достаточно чувствительна к выбросам. Давайте вернемся к нашим данным про автомобили и добавим туда выбросы.

Посмотрим, как поведет себя простая линейная регрессия.

In [ ]:
df_train = pd.read_csv('http://bit.ly/1gIQs6C')

In [ ]:
X_train = df_train.mileage.values.reshape(-1, 1)
y_train = df_train.price.values

n = y_train.shape[0]

In [ ]:
## Добавляем выбросы

In [ ]:
for i in range(10):
    X_train = np.r_[X_train, [[250000+np.random.rand()*10000]]]
    y_train = np.r_[y_train, 16000+np.random.randn()*1000]

In [ ]:
plt.scatter(X_train, y_train)

In [ ]:
model = LinearRegression(fit_intercept=True)
model.fit(X_train[:n], y_train[:n])

model_ouliers = LinearRegression(fit_intercept=True)
model_ouliers.fit(X_train, y_train)

In [ ]:
x = np.linspace(0, max(X_train), 100).reshape(-1, 1)
y_hat = model.predict(x)
y_hat_outliers = model_ouliers.predict(x)

fig, ax = plt.subplots(1, 1, figsize=(10,5))
ax.scatter(X_train, y_train)

ax.plot(x, y_hat, c='red', label='good model')
ax.plot(x, y_hat_outliers, c='green', label='biased model')
plt.legend()

### RANSAC регрессия

Идея метода RANdom SAmple Consensus (RANSAC) заключается в многократном обучении модели на случайном наборе точек из исходных данных с последующим выбором лучшей модели.

То есть:
* Задаем функцию потерь
* Задаем порог $\theta$ для остатков при котором наблюдения начинают относится к выбросам
* Задаем правило останова

Шаги алгоритма следующие
1. Взять случайные K точек и обучить на них модель M
2. Сравнить ошибки на остальных точких с порогом $\theta$ и отнести к выбросам или внутренним точкам
3. Обучить модель на всех внутренних точках, оценить качество на внутренних точках
4. Повторить 1-3 пока не наступит правило останова. 
5. Вывод: модель с лучшим качеством

In [ ]:
from sklearn.linear_model import RANSACRegressor

In [ ]:
model_ransac = RANSACRegressor(LinearRegression())
model_ransac.fit(X_train, y_train)

In [ ]:
x = np.linspace(0, max(X_train), 100).reshape(-1, 1)
y_hat = model_ransac.predict(x)

fig, ax = plt.subplots(1, 1, figsize=(10,5))
ax.scatter(X_train, y_train)

ax.plot(x, y_hat, c='red', label='RANSAC model')
ax.plot(x, y_hat_outliers, c='green', label='biased model')
plt.legend()

### Robust Estimators

Идея робастных методов заключается во взвешивании остатков модели таким образом, чтобы большие значения вносили меньший вклад в оценку параметров.

Таким образом, вместо минимизации квадрата остатков $$ L(\beta_0,\beta_1,\dots) = \frac{1}{2n}\sum^{n}_{i=1}(\hat{y}^{(i)} - y^{(i)})^2$$
Будут минимизироваться взвешенные остатки $$ L_w(\beta_0,\beta_1,\dots) = \frac{1}{2n}\sum^{n}_{i=1}\rho(\hat{y}^{(i)} - y^{(i)}),$$
где $\rho(\cdot)$ - некоторая взвешивающая функция.

Для того, чтобы попробовать эти методы нужно будет устновить пакет `statsmodels` через `pip`

In [ ]:
!pip install statsmodels

In [ ]:
import statsmodels.api as sm

In [ ]:
c = 4.685
support = np.linspace(-3*c, 3*c, 1000)
tukey = sm.robust.norms.TukeyBiweight(c=c)
plt.plot(support, tukey(support))
plt.ylim(.1, -4.1)

In [ ]:
model_robust = sm.RLM(y_train, sm.add_constant(X_train), M=sm.robust.norms.TukeyBiweight())
model_robust = model_robust.fit()

In [ ]:
model_robust.summary()

In [ ]:
x = np.linspace(0, max(X_train), 100).reshape(-1, 1)
y_hat = model_robust.predict(sm.add_constant(x))

fig, ax = plt.subplots(1, 1, figsize=(10,5))
ax.scatter(X_train, y_train)

ax.plot(x, y_hat, c='red', label='Robust model')
ax.plot(x, y_hat_outliers, c='green', label='biased model')
plt.legend()

# Логистическая регрессия

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,8)

# Для кириллицы на графиках
font = {'family': 'Verdana',
        'weight': 'normal'}
plt.rc('font', **font)

## Игрушечный пример

Сгенерируем выборку и опробуем логистическую регрессию

In [ ]:
np.random.seed(0)
X = np.r_[np.random.randn(20, 2) + [2, 2],
          np.random.randn(20, 2) + [-2, -2]]
y = [-1] * 20 + [1] * 20

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
ax.scatter(X[:, 0],
           X[:, 1],
           c=y,
           cmap=plt.cm.Paired)

In [ ]:
from sklearn.linear_model import LogisticRegression

Обучим логистическую регрессию на этих данных и нарисуем разделяющую гиперплоскость

In [ ]:
model = LogisticRegression(C=1.0, 
                           fit_intercept=True, 
                           penalty='l2')
model.fit(X, y)

In [ ]:
print 'w_0 = %f' % model.intercept_
print 'w_1, w_2 = ', model.coef_

In [ ]:
# Нарисуем эту гиперплоскость
w_0 = model.intercept_[0]
w_1 = model.coef_[0][0]
w_2 = model.coef_[0][1]

x_1 = np.linspace(-4, 4, 10)
x_2 = - (w_0 + w_1*x_1)/w_2

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
ax.scatter(X[:, 0],
           X[:, 1],
           c=y,
           cmap=plt.cm.Paired)
plt.plot(x_1, x_2)

In [ ]:
y_hat = model.predict(X)
y_hat[:10]

In [ ]:
y_hat_proba = model.predict_proba(X)
y_hat_proba[:10, :]

In [ ]:
dec_func = model.decision_function(X)
dec_func[:10]

### Как сделать нелинейную границу?

Рассмотрим набор данных, который в простонародье называют "Бублик".

In [ ]:
from sklearn.datasets import make_circles

In [ ]:
X, y = make_circles(n_samples=100, shuffle=True,
                    noise = 0.1,
                    factor=0.1)

plt.scatter(X[:, 0],
            X[:, 1],
            c=y)

Очевидно, что классы нельзя разделить линией. Но можно сделать это окружностью! </br>
Т.е. разделяющся линия теперь будет задаваться не уравнением прямой $g(x) = w_0 + w_1x_1 + w_2x_2$, а уравнением окружности. 

Выполните преобразование матрицы X, чтобы в ней были столбцы для $x_1$, $x_2$, $x^2_1$ + $x^2_2$ и обучите логистическую регрессию.

In [ ]:
# Your code Here
X_new = ...
model = LogisticRegression(C=100000, 
                           fit_intercept=True)
model.fit(X_new, y)

In [ ]:
# Посчитаем количество ошибок
y_hat = model.predict(X_new)
(y != y_hat).sum()

In [ ]:
# Нарисуем полученную окружность

x0, x1 = np.meshgrid(np.arange(-1.5, 1.5, 0.1),
                       np.arange(-1.5, 1.5, 0.1))
xx0, xx1 = x0.ravel(), x1.ravel()

# Your code here

## Анализ тональности

Загрузите текстовые данные [отсюда](https://archive.ics.uci.edu/ml/machine-learning-databases/00331/). Архив должен содержать 3 файла с положительными и отрицательными отзывами с ресурсов
* imdb.com
* amazon.com
* yelp.com

Формат файла следующий:
<отзыв>\t<метка>\n


### Задача
1. Загрузите тексты и метки классов в разные переменные
2. Выберите меру качества классификации
3. Обучите логистическую (без подбора гиперпараметров). Тексты представляются в виде мешка слов
4. Выведите наиболее значимые слова из текста

In [ ]:
sent_data = pd.read_csv('sentiment/yelp_labelled.txt', sep='\t', names=['sentence', 'label'], header=None)

In [ ]:
sent_data.head()

In [ ]:
sent_data.shape

In [ ]:
X = sent_data.sentence.values
y = sent_data.label.values

In [ ]:
# Your Code Here

## Кредитный скорринг

Рассмотрим данные по кредитованию.

Столбец с классом называется `y`.<br/> Значение $1$ соответствует классу клиентов банка, которым выдали кредит и они его успешно вернули.<br/> Значение $-1$ соответствует клиентам, невыполнившим свои кредитные обязанности. 

В банке хотят уметь определять по признакам `a1-a15`, сможет ли новый клиент вернуть кредит или нет? То есть нам надо обучить классификатор! *Обычно, в банках используют скор-карты, но процесс их построения тесно связан с логистической регрессией*

Загрузите данные и преобразуйте признаки `a1`, `a9`, `a10` и `a12` из строковых в числовые. В них только 2 возможных значения. Для этого можно использовать функцию DataFrame.replace() в `pandas` или самое обычное присваивание на соответствующих строках.

In [ ]:
df = pd.read_csv('crx.data')

df.head()

### Предобработка данных

#### Заполнение пропусков

Заполните пропуски в данных

In [ ]:
df.a1 = df.a1.fillna(value=df.a1.mode().values[0])
df.a2 = df.a2.fillna(value=df.a2.median())

df.a4 = df.a4.fillna(value=df.a4.mode().values[0])
df.a5 = df.a5.fillna(value=df.a5.mode().values[0])
df.a6 = df.a6.fillna(value=df.a6.mode().values[0])
df.a7 = df.a7.fillna(value=df.a7.mode().values[0])

df.a14 = df.a14.fillna(value=df.a14.median())

#### Преобразование признаков

В признаках `a6`, `a7` присутствуют "редкие" значение. Найдите их с помощью фунцкии `.value_counts()`  и объедините, присвоив им одно и то же значение, например `'Other'`.

In [ ]:
# Your code here
df.a7.replace({'z': 'Other', 'j': 'Other', 'dd': 'Other',
               'n': 'Other', 'o': 'Other'}, inplace=True)

df.a6.replace({'j': 'Other', 'r': 'Other'}, inplace=True)

# df.a6 = af.a6.replace({'j': 'Other', 'r': 'Other'}, inplace=True)

Влияние категориальной (да и в общем-то любой) переменной на целевую переменную можно с помощью разных метрик.

Для категорий есть мера [Normalized Mutual Information](https://en.wikipedia.org/wiki/Mutual_information)

Выделите бинарные признаки `a1`, `a9`, `a10` и `a12` в матрицу `X_binary`

In [ ]:
# Your code here
df.a1.replace({'a': 1, 'b': 0}, inplace=True)
df.a9.replace({'t': 1, 'f': 0}, inplace=True)
df.a10.replace({'t': 1, 'f': 0}, inplace=True)
df.a12.replace({'t': 1, 'f': 0}, inplace=True)

X_binary= df.loc[:, ['a1', 'a9', 'a10','a12']].values

На занятиях мы изучатли преобразование категориальных признаков с помощью комбинации `LabelEncoder` и `OneHotEncoder` или метода `pd.get_dummies`. <br\>

Есть еще один способ из библиотеки sklearn - `DictVectorizer`. Если хотите - можете разобраться, что делает код ниже, изучая промежуточные результаты каждой из команд.

In [ ]:
from sklearn.feature_extraction import DictVectorizer

In [ ]:
dv = DictVectorizer(sparse=False)
dv.fit(df[['a5', 'a6', 'a7', 'a13']].T.to_dict().values())

X_cat = dv.transform(df[['a5', 'a6', 'a7', 'a13']].T.to_dict().values())

In [ ]:
X_cat.shape

`X_cat` - эквивалентна последовательному применению `LabelEncoder` к каждому признаку `a5`, `a6`, `a7`, `a13` и использованию `OneHotEncoder`. Чтобы узнать, чему соответствует каждый столбец матрицы `X_cat` используйте команду `dv.feature_names_`

In [ ]:
dv.feature_names_

Вы долнжы увидеть список с элементами вида <Признак>=<Значение>. Если в каком-то из случаев это не так, то
* Либо `DictVectorizer` посчитал, что признак - некатегориальный
* Либо в признаке есть пропуски

С помощью метода  df[..].corr() убедитесь, что признаки `a2`, `a3`, `a8`, `a11`, `a14` и `a15` довольно слабо коррелируют друг с другом.

In [ ]:
df.loc[:, ['a2', 'a3', 'a8', 'a11', 'a14', 'a15']].corr()

In [ ]:
from sklearn.preprocessing import StandardScaler

Нормализуйте количественные признаки `a2`, `a3`, `a8`, `a11`, `a14` и `a15` с помощью `StandartScaler` или вручную. Вы должны получить матрицу `X_real`.

**МЕТОДОЛОГИЧЕСКИЙ КОСЯК:** нужно сначала разбить на трейн и тест, а потом использовать всякое шкалирование и кодирование. Но для целей этого задания - это не так вредно.

In [ ]:
scaler = StandardScaler()
X_real = scaler.fit_transform(df[['a2', 'a3', 'a8', 'a11', 'a14', 'a15']].values)

Используйте функцию `np.concatinate(..)` или `np.c[..]` чтобы сцепить матрицы `X_binary`, `X_cat` и `X_real`

В результате вы должны получить матрицу с преобразованными призанками `X` и вектор ответов `y`

In [ ]:
X = np.c_[X_binary, X_cat, X_real]
y = df.loc[:, 'y'].values

In [ ]:
X.shape

### Регуляризация

Помимо мультиколлинеарности логистическая регрессия так же может быть подвержена переобучению. Как правило, на деле это выражается в 
* Очень больших весах $|w_i|$ при признаках
* Неустойчивости решения (особенно в случае мультиколлинеарности, когда возникает линейная зависимость признаков и получается целое семейство равнозначных моделей)
* Большая разница в качестве на обучении и валидации

Регуляризация - это техника понижения "сложности" модели с минимальными последствиями для её качества.

В случае с логистичесткой регресии, сложность модели выражается в значениях весов $w_j$ при признаках. Больший вес означает большее влияние признака на результат. В таком случае, давайте добавил штрафное слагаемое в функцию оптимизации для логистической регресии. Самый распространенные из них это:

Стало (Ridge Regularization)
$$ L(\beta_0,\beta_1,\dots) = \frac{1}{2n}\left[ \sum^{n}_{i=1}(a(x^{(i)}) - y^{(i)})^2\right] + \frac{1}{C}\sum_{j=1}^{m}\beta_j^2 $$
или (Lasso Regularization)
$$ L(\beta_0,\beta_1,\dots) = \frac{1}{2n}\left[ \sum^{n}_{i=1}(a(x^{(i)})  - y^{(i)})^2\right] + \frac{1}{C}\sum_{j=1}^{m}|\beta_j| $$

$C$ - называется гиперпараметром регуляризации и он задается вручную. Выбирается он с помощью кросс-валидации. Чем больше $С$ - тем меньше влияние регуляризации.

Lasso regression называется так, потому что она осуществляет "отлов" признаков - т.е. незначимые признаки будут иметь нулевые веса в модели, в то время как в Ridge regression - веса будут постепенно падать у всех признаков.

<img src='http://webdancer.is-programmer.com/user_files/webdancer/Image/lasso.png'>

Однако использование регуляризации превращает логистическую регрессию в черный ящик - мы предоставляем оптимизационному методу решать за нас, какие признаки важны для модели.

Давайте сравним работу регуляризаторов. 

1. Разбейте данные на обучающую и контрольную выборки.
1. Для $C$ из набора np.logspace(-3, 3, 10) обучите LogisctigRegression c Lasso регуляризацией (`penalty='l1'`). На каждой итерации оцените качество (ROC-AUC) на контрольной выборке и запомните полученные коэффициенты модели
1. На одном графике выведите значение качества в зависимости от параметра `C` 
1. На другом графике для каждого признака выведите изменение коэффициента в модели в зависимости от параметра `C`
1. Для оптимальной на ваш взгляд настройки модели выведите 5 наиболие "важных" признаков и их коэффициенты
1. Проделайте тоже самое для Ridge регуляризации (`penalty='l2'`)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [ ]:
model = LogisticRegression(penalty='l1', C=1, fit_intercept=True)
#... После fit()

# # Коэффициенты w_1...w_d
# model.coef_

# # Коэффициент при свободном члене w_0
# model.intercept_

# # Предсказание
# model.predict(X_test)
# model.predict_proba(X_test)
# model.decision_function(X_test)


In [ ]:
coefs = np.empty((X.shape[1],))
scores = []

c_range = np.logspace(-3, 3, 10)

for C in c_range:
# Your Code Here    

## Tree embedding

Вот так (примерно) выглядит скор-карта на выдачу кредита

<img src='https://www.mathworks.com/help/finance/credit_score_card_overview.png'>

Каждый "признак" разбит на бины (bins) - значимые интервалы. Если значение признака попадает в интервал, то к скору заемщика прибавляется сооветвтвующая величина.

Таким образом формируется итоговый скор и далее принимается решение о выдаче кредита.

Оценка порого бинов для признаков - это отдельная наука. Скоррингисты используют так называемое IV (information value) для оценки качества разбиения признака на бины.

На прошлом занятии мы изучили с вами другой способ выявления порогов на признаках - деревья решений (случайный лес). Почему бы нам это не использовать

### Задание
* Разбейте выборку на 2 части (обучение и контроль)
* На половине обучающей выборке обучите случайный лес
* С помощью метода `rand_forest.apply(X)` можно получить матрицу размера $n \times t$, где $t$ - количество деревьев в лесу. В эту матрицу будет числом записан номер листа, в который попал объект
* One-Hot Encoding!
* Обучите логистическую регрессию на второй половине обучающей выборки
* Сравние roc-auc на обычной лог-регрессии, обычного случайного леса и их композиции

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X_train2, X_emb, y_train2, y_emb = train_test_split(X_train, y_train, 
                                                    test_size = 0.5, random_state=123)

In [ ]:
rand_forest = RandomForestClassifier(n_estimators=100, max_depth=3)
rand_forest.fit(X_emb, y_emb)

X_train2_emb = rand_forest.apply(X_train2)
X_test_emb = rand_forest.apply(X_test)

In [ ]:
plt.imshow(X_train2_emb)

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
# Комбо модель
model1 = Pipeline([('onehot', OneHotEncoder(handle_unknown='ignore')),
                  ('logreg', LogisticRegression())])
model1.fit(X_train2_emb, y_train2)

# Простой лес
model2 = RandomForestClassifier(n_estimators=100, max_depth=3)
model2.fit(X_train, y_train)

# Простая лог регрессия
model3 = LogisticRegression()
model3.fit(X_train, y_train)


In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [ ]:
for i, model in enumerate([model1, model2, model3]):
    try:
        y_hat = model.predict_proba(X_test_emb)[:, 1]
    except:
        y_hat = model.predict_proba(X_test)[:, 1]
        
    fpr, tpr, thresholds = roc_curve(y_test, y_hat)
    
    plt.plot(fpr, tpr, label='model%d' % (i+1))
    
plt.legend()

